In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [2]:
temp = pd.read_excel('D12_D16_Distance_Index.xlsx',header=0)
#Distance of paath from D12
dist_d12 = temp['d_D12']
#Distance of paath from D16
dist_d16 = temp['d_D16']
#Transmitter
tx = temp['Tx']
#Receiver
rx = temp['Rx']
#Index
index = temp['Serial Index']
temp.head(5)

,PathIndex,Tx,Rx,d_D16,d_D12,Serial Index
0,5,1,7,115.00,255.00,0
1,6,1,8,75.31,225.84,1
2,7,1,9,36.31,191.81,2
3,8,1,10,0.39,156.82,3
4,9,1,11,30.51,123.88,4


In [4]:
path = 'E:\Shruti\PythonCodes'

for t in np.arange(10,101,10):
    ind_d16 = []
    txn =[]
    rxn =[]
    for i in range(36):
        if dist_d16[i] <= t:
            ind_d16.append(i)
            txn.append(tx[i])
            rxn.append(rx[i])

    for k in np.arange(1,5):
        mse = []
        auc_d16 =[]
        
        #path names
        for i in np.arange(1,7):
            for j in np.arange(7,13):
                mse.append(path+'\P'+str(i)+'-'+str(j)+'_MSE_temp_case'+str(k)+'.csv')

        for i in range(len(ind_d16)):
            #read mse files
            data = np.array(pd.read_csv(mse[ind_d16[i]],header=0))
            mse_bl = data[:,0]
            mse_D12 = data[:,1]
            mse_D16 = data[:,2]
            
            #targets
            targets_all =  ([1] * len(mse_D16)) +([0] * len(mse_bl)*2)
            
            #predictions
            prediction_all = np.concatenate((mse_D16,mse_D12,mse_bl), axis=0)

            fpr, tpr, _ = roc_curve(targets_all, prediction_all)
            roc_auc = auc(fpr, tpr)
            
            #append score
            auc_d16.append(auc(fpr, tpr))
        
        #save the auc in excel files
        auc_d16 = np.array(auc_d16)
        txn = (np.array(txn)).reshape(len(auc_d16),1)
        rxn = (np.array(rxn)).reshape(len(auc_d16),1)
        temp = auc_d16.reshape(len(auc_d16),1)
        df_features = pd.DataFrame(data = np.concatenate([txn,rxn,temp],axis=1),columns = ['Tx','Rx','AUC'])
        df_features.to_excel(path+'\AUC_D16_temp_variation_case'+str(k)+'_thresh'+str(t)+'.xlsx', index=False)


In [6]:
df_features.head(5)

,Tx,Rx,AUC
0,1,12,0.625
1,2,11,0.500
2,2,12,1.000
3,3,9,0.500
4,3,10,0.875
